In [1]:
import numpy as np

In [2]:
def create_profile_matrix(motifs): #laplace
    profile = []
    for j in range(len(motifs[0])):
        counts = np.array([0, 0, 0, 0])
        for i in range(len(motifs)):
            if (motifs[i][j] == 'A'):
                counts[0] += 1
            elif (motifs[i][j] == 'C'):
                counts[1] += 1
            elif (motifs[i][j] == 'G'):
                counts[2] += 1
            else: # T
                counts[3] += 1
            counts = counts + 1
        profile.append(counts/(counts[0] + counts[1] + counts[2] + counts[3]))
    return np.transpose(profile)

In [6]:
def compute_pr(text, profile):
    product = 1
    i = 0
    for nucleotide in text:
        #print(nucleotide)
        if (nucleotide == 'A'):
            product *= profile[0,i]
        elif (nucleotide == 'C'):
            product *= profile[1,i]
        elif (nucleotide == 'G'):
            product *= profile[2,i]
        else:
            product *= profile[3,i]
        i += 1
    return product

In [7]:
def profile_most_probable_kmer(text, k, profile):
    probabilities = []
    for i in range(len(text)-k+1):
        prob = compute_pr(text[i:i+k], profile)
        probabilities.append(prob)
    j = probabilities.index(max(probabilities))
    return text[j:j+k]

In [8]:
def motifs(profile, Dna):
    kmer_collection = []
    for strand in Dna:
        kmer_collection.append(profile_most_probable_kmer(strand, len(profile[0]), profile))
    return kmer_collection

In [9]:
def hamming_dist(str1, str2): #assume equal length
    '''
    The number of mismatches between strings p and q
    is called the hamming distance between these
    strings and is denoted hamming_dist(p, q).

    Input: Two strings of equal length.
    Output: The Hamming distance between these
    strings.
    '''
    dist = 0
    for i in range(len(str1)):
        if (str1[i] != str2[i]):
            dist +=1
    return dist

In [10]:
def score(motifs):
    '''
    Define score(motifs) as the number of unpopular
    letters/nucleotides in a motif matrix 'motifs'.
    '''
    consensus = ''
    profile = create_profile_matrix(motifs)
    profile_len = len(profile[0])
    for i in range(profile_len):
        maximum = max(profile[:,i])
        if (maximum == profile[0,i]):
            consensus += 'A'
        elif (maximum == profile[1,i]):
            consensus += 'C'
        elif (maximum == profile[2,i]):
            consensus += 'G'
        else: #T
            consensus += 'T'
    score = 0
    for motif in motifs:
        score += hamming_dist(consensus, motif)
    return score

In [28]:
def randomized_motif_search(Dna, k, t):
    '''
    Input: Integers k and t, followed by a collection
    of strings Dna.
    Output: A collection BestMotifs.
    '''
    motif_list = []
    for strand in Dna:
        rand_index = np.random.randint(0, len(strand) - k + 1)
        motif_list.append(strand[rand_index:rand_index + k])
    best_motifs = motif_list[:]
    while 0 < 1:
        profile = create_profile_matrix(motif_list)
        motif_list = motifs(profile, Dna)
        if score(motif_list) < score(best_motifs):
            best_motifs = motif_list[:]
        else:
            return best_motifs

In [95]:
def randomized_motif_search_quiz(Dna, k, t):
    motif_list = ['GTC','CCC','ATA','GCT']
    best_motifs = motif_list[:]
    i = 0
    while 0 < 1:
        print(i)
        profile = create_profile_matrix(motif_list)
        motif_list = motifs(profile, Dna)
        print(motif_list)
        if score(motif_list) < score(best_motifs):
            best_motifs = motif_list[:]
            i += 1
        else:
            return best_motifs

In [13]:
def run_randomized_motif_search_1000(Dna, k, t):
    '''
    Input: Integers k and t, followed by a collection
    of strings Dna.
    Output: A collection BestMotifs.resulting from
    running RandomizedMotifSearch(Dna, k, t) 1,000
    times.
    '''
    last_motifs = randomized_motif_search(Dna, k, t)
    for i in range(1000):
        best_motifs = randomized_motif_search(Dna, k, t)
        if score(best_motifs) < score(last_motifs):
            last_motifs = best_motifs[:]
    return last_motifs

In [35]:
from random import choice
from random import choices

In [262]:
def profile_randomly_generated_kmer(text, k, profile):
    probabilities = []
    for i in range(len(text)-k+1):
        prob = compute_pr(text[i:i+k], profile)
        probabilities.append(prob)
    j = choices(range(len(probabilities)), weights=probabilities/sum(probabilities))[0]
    return text[j:j+k]

In [1]:
def gibbs_sampler(Dna, k, t, N):
    '''
    GibbsSampler uses this random number generator to
    select a Profile-randomly generated kmer at each
    step: if the die rolls the number i, then we
    define the Profile-randomly generated k-mer as
    the i-th k-mer in Text.

    Input: Integers k, t, and N, followed by a 
    collection of strings Dna.
    Output: The strings BestMotifs resulting from
    running GibbsSampler(Dna, k, t, N).
    '''
    motif_list = []
    for strand in Dna:
        rand_index = np.random.randint(0, len(strand) - k + 1)
        motif_list.append(strand[rand_index:rand_index + k])
    best_motifs = motif_list[:]
    for j in range(N):
        i = np.random.randint(0,t)
        motif_list.pop(i)
        profile = create_profile_matrix(motif_list)
        motif_list.insert(i, profile_randomly_generated_kmer(Dna[i], k, profile))
        if score(motif_list) < score(best_motifs):
            best_motifs = motif_list[:]
    return best_motifs

In [85]:
def run_gibb_sampler_20(Dna, k, t, N):
    '''
    Runs gibbs sampler with 20 random
    starts.
    '''
    last_motifs = gibbs_sampler(Dna, k, t, N)
    for i in range(20):
        best_motifs = gibbs_sampler(Dna, k, t, N)
        if score(best_motifs) < score(last_motifs):
            last_motifs = best_motifs[:]
            print(score(best_motifs))
    return last_motifs